# Energy Reconstruction Using CNN

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Attention

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

In [4]:
# Name for model
key = 'CNN_attn'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True} #i dont understand exactly what all of these options are

In [5]:
x, y = load_preprocessed(simPrefix, 'train')
print(y.keys())

Percentage of events with a NaN: 2.68
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


In [10]:
from tensorflow.keras import Input, layers, Model
import tensorflow as tf

inputs = Input(shape=(10,10,2))

#o = layers.Attention()([inputs, inputs]) #.1336 loss compared to .1274 -- WORSE ( Model = 1 layer, then pooling, conv)

#Loss on Conv-Conv-Conv = 0.0522

#print(tf.shape(o))
o = layers.Attention()([inputs, inputs]) #.1336 loss compared to .1274
o = layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2))(o)
#o = MaxPooling2D(pool_size = (2,2), strides = None)(inputs)
o = layers.Attention()([o, o]) #.1336 loss compared to .1274
o = layers.Conv2D(32, kernel_size=3, activation='relu')(o)
#o = MaxPooling2D(pool_size = (2,2), strides = None)(o)
o = layers.Attention()([o,o]) #.1336 loss compared to .1274
o = layers.Conv2D(16, kernel_size=3, activation='relu')(o)

o = layers.Flatten()(o)
outputs = layers.Dense(10)(o)


model = Model(inputs=inputs, outputs=outputs, name="attn_test")

model.summary()

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])




Model: "attn_test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10, 10, 2)]  0                                            
__________________________________________________________________________________________________
attention_1 (Attention)         (None, 10, 10, 2)    0           input_2[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 64)     1216        attention_1[0][0]                
__________________________________________________________________________________________________
attention_2 (Attention)         (None, 8, 8, 64)     0           conv2d[0][0]             

In [11]:
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy


In [12]:
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=3)



Epoch 1/3
14592/14592 [==============================] - 102s 7ms/step - loss: 0.2055 - mse: 0.2055 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 2/3
14592/14592 [==============================] - 101s 7ms/step - loss: 0.0980 - mse: 0.0980 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 3/3
14592/14592 [==============================] - 100s 7ms/step - loss: 0.0890 - mse: 0.0890 - val_loss: 0.1174 - val_mse: 0.1174


In [13]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)